In [ ]:
# Importing the necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Bidirectional, Input, BatchNormalization, CuDNNGRU, CuDNNLSTM
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints
from keras.utils.vis_utils import plot_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [ ]:
x_train = pd.read_csv(r'/content/Train_Features.csv', index_col=0)
y_train = pd.read_csv(r'/content/Train_Labels.csv', index_col=0)
x_test = pd.read_csv(r'/content/Test_Features.csv', index_col=0)
y_test = pd.read_csv(r'/content/Test_Labels.csv', index_col=0)

In [ ]:
categories = list(y_train.columns)
print(categories)
print(y_test.sum())

In [ ]:
lstm_out = 196

model = Sequential()
model.add(Embedding(1000, 32, input_length = x_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.5))
model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
callbacks = [
    ReduceLROnPlateau(), 
    EarlyStopping(patience=4), 
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(x_test, y_test),
                    batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))